In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import seaborn as sns
import matplotlib.pyplot as plt

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 3, Finished, Available, Finished)

In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
model_path = "/lakehouse/default/Files/Models/best_voucher_model.pkl"
if not os.path.exists(model_path):
    raise FileNotFoundError("❌ Không tìm thấy model. Hãy chạy notebook Train_Model trước!")
model = joblib.load(model_path)
print("✅ Model đã được load thành công từ Lakehouse!")

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 4, Finished, Available, Finished)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [23]:
df_2023 = spark.sql("""
SELECT 
    f.BillID,
    f.Channel,
    f.OrderFrom,
    f.TransactionDate,
    f.SalesAmount,
    f.CustomerID,
    f.Store_ID,
    c.CustomerGender,
    s.Region
FROM FabricStaging.fact_sales f
LEFT JOIN FabricStaging.dim_customer c
    ON f.CustomerID = c.CustomerID
LEFT JOIN FabricStaging.dim_store s
    ON f.Store_ID = s.Store_ID
WHERE year(TransactionDate) = 2023
""").toPandas()
print(f"📊 Dữ liệu năm 2023: {df_2023.shape[0]} dòng, {df_2023.shape[1]} cột")

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 25, Finished, Available, Finished)

📊 Dữ liệu năm 2023: 19508 dòng, 9 cột


In [24]:
df_2023['TransactionDate'] = pd.to_datetime(df_2023['TransactionDate'], errors='coerce')
df_2023['Year'] = df_2023['TransactionDate'].dt.year
df_2023['Month'] = df_2023['TransactionDate'].dt.month
df_2023['Quarter'] = df_2023['TransactionDate'].dt.quarter
df_2023['DayOfWeek'] = df_2023['TransactionDate'].dt.day_name()
df_2023.head()

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 26, Finished, Available, Finished)

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,Store_ID,CustomerGender,Region,Year,Month,Quarter,DayOfWeek
0,1029067,Delivery,CALL CENTER,2023-01-01 00:00:00+00:00,375012,1452379,S0425,Unknown,Hanoi,2023,1,1,Sunday
1,1029068,Delivery,APP,2023-01-01 00:00:00+00:00,493194,526953,S0391,Male,Southern Provinces,2023,1,1,Sunday
2,1029069,Take Away,STORE,2023-01-01 00:00:00+00:00,81748,1033262,S0367,Unknown,Hanoi,2023,1,1,Sunday
3,1029070,Take Away,STORE,2023-01-01 00:00:00+00:00,292970,106225,S0496,Unknown,Hanoi,2023,1,1,Sunday
4,1029071,Delivery,WEBSITE,2023-01-01 00:00:00+00:00,528095,212809,S0451,Unknown,Hanoi,2023,1,1,Sunday


In [25]:
features = ['Channel', 'OrderFrom', 'CustomerGender', 'Region', 'SalesAmount', 'Year', 'Month', 'DayOfWeek']
X_2023 = pd.get_dummies(df_2023[features], drop_first=True)
print("🧩 Dữ liệu đầu vào đã chuẩn hoá khớp với model train.")

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 27, Finished, Available, Finished)

🧩 Dữ liệu đầu vào đã chuẩn hoá khớp với model train.


In [27]:
# 🤖 5️⃣ Dự đoán xác suất và nhãn
# =========================================================
df_2023['Voucher_Prob'] = model.predict_proba(X_2023)[:, 1]
df_2023['Voucher_Predict'] = (df_2023['Voucher_Prob'] > 0.5).astype(int)
df_2023['Voucher_Predict_Label'] = df_2023['Voucher_Predict'].map({1: 'Yes', 0: 'No'})

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 29, Finished, Available, Finished)

In [18]:
df_2023

StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 20, Finished, Available, Finished)

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,Store_ID,CustomerGender,Region,Year,Month,Quarter,DayOfWeek,Voucher_Prob,Voucher_Predict,Voucher_Predict_Label
0,1029067,Delivery,CALL CENTER,2023-01-01 00:00:00+00:00,375012,1452379,S0425,Unknown,Hanoi,2023,1,1,Sunday,0.152679,0,No
1,1029068,Delivery,APP,2023-01-01 00:00:00+00:00,493194,526953,S0391,Male,Southern Provinces,2023,1,1,Sunday,0.760549,1,Yes
2,1029069,Take Away,STORE,2023-01-01 00:00:00+00:00,81748,1033262,S0367,Unknown,Hanoi,2023,1,1,Sunday,0.001793,0,No
3,1029070,Take Away,STORE,2023-01-01 00:00:00+00:00,292970,106225,S0496,Unknown,Hanoi,2023,1,1,Sunday,0.163282,0,No
4,1029071,Delivery,WEBSITE,2023-01-01 00:00:00+00:00,528095,212809,S0451,Unknown,Hanoi,2023,1,1,Sunday,0.205084,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19503,1048570,Delivery,STORE,2023-01-09 00:00:00+00:00,178107,1398136,S0460,Unknown,Southern Provinces,2023,1,1,Monday,0.014268,0,No
19504,1048571,Take Away,STORE,2023-01-09 00:00:00+00:00,331447,2023258,S0499,Unknown,Hanoi,2023,1,1,Monday,0.219202,0,No
19505,1048572,Take Away,WEBSITE,2023-01-09 00:00:00+00:00,381509,2041510,S0426,Female,Southern Provinces,2023,1,1,Monday,0.910378,1,Yes
19506,1048573,Take Away,STORE,2023-01-09 00:00:00+00:00,508205,1927215,S0393,Unknown,Hanoi,2023,1,1,Monday,0.206322,0,No


In [30]:
# Chuyển pandas DataFrame sang Spark DataFrame
df_spark = spark.createDataFrame(df_2023)
# Ghi vào Lakehouse (bảng)
df_spark.write.mode("overwrite").format("delta").saveAsTable("Prediction_VoucherStatus_2023")
print("✅ Đã lưu vào Lakehouse Tables: Prediction_VoucherStatus_2023")


StatementMeta(, 42e4073b-4ab8-4f00-b4da-a7b49f204bdb, 32, Finished, Available, Finished)

✅ Đã lưu vào Lakehouse Tables: Prediction_VoucherStatus_2023
